## Media Files

Media files are generaly stored in twi distinct ways : 
   * By downloading directly the file.
   * By reference (storing the url where the file is located)
   
Each approch has its advantages and disadvantages, the final choice on the context of the application.
    


### Download media files from a Website
Let's write a little fuction that we can use to download a specific image from a website

In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, urlretrieve

url = 'http://www.wonaruto.com/'
path = 'h1 > a > img'

def download_image(url, path):
    html = urlopen(url)
    bs = BeautifulSoup(html, 'html.parser')
    imageLocation = bs.select_one(path)['src']
    urlretrieve (imageLocation, 'logo.jpg')

download_image(url, path)


This works well if you need to download only a single files. But most scrapers don’t download a single file and call
it a day. The following downloads all internal files, linked to by any tag’s src attribute, from the home page of http://pythonscraping.com:

In [2]:
import os

downloadDirectory = 'downloaded'
baseUrl = 'http://pythonscraping.com'

def getAbsoluteURL(baseUrl, source):
    if source.startswith('http://www.'):
        url = 'http://{}'.format(source[11:])
    elif source.startswith('http://'):
        url = source
    elif source.startswith('www.'):
        url = source[4:]
        url = 'http://{}'.format(source)
    else:
        url = '{}/{}'.format(baseUrl, source)
    if baseUrl not in url:
        return None
    return url

def getDownloadPath(baseUrl, absoluteUrl, downloadDirectory):
    path = absoluteUrl.replace('www.', '')
    path = path.replace(baseUrl, '')
    path = downloadDirectory+path
    directory = os.path.dirname(path)

    if not os.path.exists(directory):
        os.makedirs(directory)

    return path

html = urlopen('http://www.pythonscraping.com')
bs = BeautifulSoup(html, 'html.parser')
downloadList = bs.findAll(src=True)

for download in downloadList:
    fileUrl = getAbsoluteURL(baseUrl, download['src'])
    if fileUrl is not None:
        print(fileUrl)

urlretrieve(fileUrl, getDownloadPath(baseUrl, fileUrl, downloadDirectory))

http://pythonscraping.com/https://pythonscraping.com/wp-includes/js/jquery/jquery.min.js?ver=7c03884fd15f65fbf9063b08180d457d
http://pythonscraping.com/https://pythonscraping.com/wp-includes/js/jquery/jquery-migrate.min.js?ver=7c03884fd15f65fbf9063b08180d457d
http://pythonscraping.com/https://pythonscraping.com/wp-content/plugins/pagelayer/js/combined.js?ver=7c03884fd15f65fbf9063b08180d457d
http://pythonscraping.com/https://pythonscraping.com/wp-content/uploads/2021/08/home1.jpg
http://pythonscraping.com/https://pythonscraping.com/wp-content/uploads/2021/08/home2.jpg
http://pythonscraping.com/https://pythonscraping.com/wp-content/uploads/2021/08/work8.jpg
http://pythonscraping.com/https://pythonscraping.com/wp-content/uploads/2021/08/work8.jpg
http://pythonscraping.com/https://pythonscraping.com/wp-content/uploads/2021/08/work8.jpg
http://pythonscraping.com/https://pythonscraping.com/wp-content/uploads/2021/08/work1.jpg
http://pythonscraping.com/https://pythonscraping.com/wp-content/up

OSError: [WinError 123] La syntaxe du nom de fichier, de répertoire ou de volume est incorrecte: 'downloaded/https:'

## Python & MySQL

We'll use the `PyMySQL` package to acces our previously defined database from a python script


In [7]:
import pymysql

# set creditentials data
password='Fwtk&SSPSb2CjBt^wYd7'
host = '127.0.0.1'

# make a connection to the SQL databse
connection = pymysql.connect(host=host, user='root', passwd=password, db='mysql')
cur = connection.cursor()


# select the database to use
cur.execute('USE scraping')

# execute a SQL request
cur.execute('SELECT * FROM pages WHERE id=1')
print(cur.fetchone())

# close connection
cur.close()


(1, 'Test page title', 'This is some test page content. It can be up to 10,000 characters\nlong.', datetime.datetime(2021, 12, 3, 11, 45, 22))


Write a simple program that scrapes data from wikipedia and insert it inside our database : 

In [1]:
import re
import random
import datetime
import pymysql

from urllib.request import urlopen
from bs4 import BeautifulSoup


# set creditentials data
password='Fwtk&SSPSb2CjBt^wYd7'
host = '127.0.0.1'

# make a connection to the SQL databse
connection = pymysql.connect(host=host, user='root', passwd=password, db='mysql', charset='utf8')
cur = connection.cursor()
cur.execute('USE scraping')

random.seed(datetime.datetime.now())


def store(title, content):
    """ takes in 2 string variables, title and content, and adds them to an INSERT statment to the PAGES table"""
    cur.execute('INSERT INTO pages (title, content) VALUES ("%s", "%s")', (title, content))
    cur.connection.commit()

    
def getLinks(articleUrl):
    """ takes a wikipedia link, extract it title and content, call store function, return a lsit of 
        all other wiki's articles links found on this page"""
    html = urlopen('http://en.wikipedia.org'+articleUrl)
    bs = BeautifulSoup(html, 'html.parser')
    title = bs.find('h1').get_text()
    content = bs.find('div', {'id':'mw-content-text'}).find('p').get_text()
    store(title, content)
    return bs.find('div', {'id':'bodyContent'}).findAll('a', href=re.compile('^(/wiki/)((?!:).)*$'))

links = getLinks('/wiki/Kevin_Bacon')
try:
    while len(links) > 0:
         newArticle = links[random.randint(0, len(links)-1)].attrs['href']
         print(newArticle)
         links = getLinks(newArticle)
finally:
    cur.close()
    conn.close()

/wiki/MTV_Movie_%26_TV_Awards
/wiki/Bill_Daily
/wiki/United_Press_International
/wiki/Agenzia_Nazionale_Stampa_Associata
/wiki/Bresciaoggi
/wiki/Gazzetta_di_Parma
/wiki/MF_Milano_Finanza
/wiki/La_Tribuna_di_Treviso
/wiki/Gazzetta_del_Sud
/wiki/Il_Giornale
/wiki/Milan
/wiki/Media_(communication)
/wiki/Information
/wiki/Ed_Hawkins_(climatologist)
/wiki/National_Centre_for_Atmospheric_Science
/wiki/Government_of_the_United_Kingdom
/wiki/Bank_of_England
/wiki/Big_City_Plan
/wiki/Economy_of_Birmingham
/wiki/Architecture_of_Birmingham
/wiki/Ian_Simpson_(architect)
/wiki/Foster_and_Partners
/wiki/Thames_Hub_integrated_infrastructure_vision
/wiki/Kent_International_Airport
/wiki/Sky_Wings
/wiki/Athens_International_Airport
/wiki/Geographic_coordinate_system
/wiki/Crustal_deformation
/wiki/ISBN_(identifier)
/wiki/ISO/IEC_9995
/wiki/ISO/IEC_42010
/wiki/ISO_518
/wiki/Hyperfocal_distance
/wiki/F-number
/wiki/Digital_image_processing
/wiki/Luminance
/wiki/Lambert_(unit)
/wiki/Skot_(unit)
/wiki/Foot

NameError: name 'conn' is not defined

In [6]:
cur.execute('SELECT title FROM pages')
print(cur.fetchone())


('Test page title',)
